# RAG

Para este ejemplo se mejorará el ejemplo que se creó antes ([aquí](../01-rag/rag.ipynb)), se utilizará la misma fuente de información, pero ahora siguiendo el correcto paso a paso de cómo se debería hacer en realidad (ver [aquí](../rag-explanation.md)). Para este caso, utilizaremos la libreria `langchain` con Claude como LLM, y bases de datos de vectores para almacenar la información de los documentos

## Importar módulos y todo lo necesario

Para el modelo de embedding se puede cambiar a otro en caso de ser necesario

In [37]:
# LLM
from langchain_anthropic import ChatAnthropic

# Para el indexing
from langchain_core.documents import Document
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_nomic import NomicEmbeddings
from langchain_chroma import Chroma

# Para la generación
from langchain import hub

# Variables de entorno
from os import getenv
from dotenv import load_dotenv

load_dotenv()

CLAUDE_API_KEY = getenv("CLAUDE_API_KEY")
MODEL_NAME = getenv("CLAUDE_MODEL_NAME")
NOMIC_API_KEY = getenv("NOMIC_API_KEY")
LANGCHAIN_API_KEY = getenv("LANGCHAIN_API_KEY")

CHUNK_SIZE = int(getenv("CHUNK_SIZE"))
CHUNK_OVERLAP = int(getenv("CHUNK_OVERLAP"))

In [20]:
llm: ChatAnthropic = ChatAnthropic(
    api_key=CLAUDE_API_KEY,
    model=MODEL_NAME,
    temperature=0,
    max_tokens=2000,
)

text_splitter: RecursiveCharacterTextSplitter = RecursiveCharacterTextSplitter(
    chunk_size=CHUNK_SIZE, chunk_overlap=CHUNK_OVERLAP
)

embeddings: NomicEmbeddings = NomicEmbeddings(
    model="nomic-embed-text-v1.5",
    nomic_api_key=NOMIC_API_KEY
)

vector_store: Chroma = Chroma(embedding_function=embeddings)

prompt = hub.pull("rlm/rag-prompt", api_key=LANGCHAIN_API_KEY)

## Indexación

### Document Loader

In [21]:
loader: TextLoader = TextLoader("./info.md", "utf-8")
docs = loader.load()

### Text Splitter

In [22]:
all_splits: list[Document] = text_splitter.split_documents(docs)
all_splits[:3]

[Document(metadata={'source': './info.md'}, page_content='# NEBULA\n\n## Roadmap\n\n### Purpose and Scope of the SAD\n\nEl SAD tiene como objetivo documentar la arquitectura de Nebula, detallando los componentes estructurales y sus interacciones para cumplir con los requisitos de desarrollo, escalabilidad y seguridad. En este documento se incluye toda la informaci�n sobre la arquitectura de software de Nebula, destacando aquellos aspectos que influyen en el rendimiento y la adaptabilidad del sistema. Las decisiones de dise�o documentadas en el SAD son aquellas que tienen un impacto arquitect�nico relevante en la estructura general del sistema, mientras que los detalles de implementaci�n espec�ficos se documentan en otros informes t�cnicos.\n\n### How the SAD Is Organized'),
 Document(metadata={'source': './info.md'}, page_content='El SAD de Nebula est� estructurado en las siguientes secciones:  \nSección 1: Documentation Roadmap - Proporciona una visi�n general del SAD, el prop�sito de

### Vector Store

In [23]:
store = vector_store.add_documents(all_splits)

In [24]:
store[:3]

['a276c71b-9590-4411-b1c2-fcdf16d105e1',
 'fc2f7bb7-fcc9-489c-84cc-11d7dfb6f65c',
 '8b08e589-1098-4cf4-8167-ba645139765e']

## Recuperación y Generación

Con `"rlm/rag-prompt"`, se está haciendo este prompt al modelo:

```
You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.
Question: {question} 
Context: {context} 
Answer:
```

Ver [esto](https://smith.langchain.com/hub/rlm/rag-prompt) para más info


En este caso, para la generación de la respuesta, no se utilizará LangGraph, pero también es posible

In [36]:
def generate(question: str):
    retrieved_docs = vector_store.similarity_search(question)
    docs_content = "\n\n".join(doc.page_content for doc in retrieved_docs)
    message_prompt = {"question": question, "context": docs_content}
    messages = prompt.invoke(message_prompt)
    response = llm.invoke(messages)
    return {**message_prompt, "answer": response.content}